# TotalSegmentator Training Experiments\n\nThis notebook trains all architectures on TotalSegmentator dataset:\n1. UNet + TotalSegmentator\n2. UNETR + TotalSegmentator\n3. SegResNet + TotalSegmentator\n\n**Prerequisites:**\n- Run `00_environment_setup.ipynb` first\n- Ensure GPU is available and TotalSegmentator dataset is uploaded\n\n**Expected Duration:** ~90 minutes total (3 experiments)

In [ ]:
# Quick verification that environment is ready\nimport torch\nimport sys\nimport os\nfrom pathlib import Path\nfrom datetime import datetime\nimport subprocess\n\nprint('Pre-training verification:')\nprint('-' * 30)\n\n# Check GPU\nif torch.cuda.is_available():\n    print(f'GPU: {torch.cuda.get_device_name(0)}')\nelse:\n    print('WARNING: No GPU detected - training will be slow')\n\n# Determine dataset path\nis_colab = 'google.colab' in sys.modules or os.path.exists('/content')\ndatasets_root = Path('/content/drive/MyDrive/datasets' if is_colab else Path.home() / 'Downloads/datasets')\n\n# Check TotalSegmentator dataset\nts_path = datasets_root / 'TotalSegmentator'\nif ts_path.exists():\n    case_count = len([d for d in ts_path.iterdir() if d.is_dir()])\n    print(f'TotalSegmentator: Available ({case_count} cases)')\nelse:\n    print('TotalSegmentator: MISSING')\n    print('Upload TotalSegmentator dataset to proceed')\n\nprint('\nReady for TotalSegmentator experiments!')

## UNet + TotalSegmentator Training

In [ ]:
print(f'Starting UNet + TotalSegmentator at {datetime.now().strftime(\"%H:%M:%S\")}')\nprint('Expected duration: ~30 minutes')\nprint('=' * 50)\n\nsubprocess.run([\n    'python', 'scripts/train_model.py',\n    '--dataset', 'totalsegmentator',\n    '--data_root', str(datasets_root),\n    '--architecture', 'unet',\n    '--in_channels', '1',\n    '--out_channels', '11',  # background + 10 major structures\n    '--batch_size', '1',     # reduced for memory efficiency\n    '--max_epochs', '50',    # reduced for feasibility\n    '--output_dir', 'results/unet_totalsegmentator'\n], check=True)\n\nprint('UNet + TotalSegmentator completed!')

## UNETR + TotalSegmentator Training

In [ ]:
print(f'Starting UNETR + TotalSegmentator at {datetime.now().strftime(\"%H:%M:%S\")}')\nprint('Expected duration: ~35 minutes')\nprint('=' * 50)\n\nsubprocess.run([\n    'python', 'scripts/train_model.py',\n    '--dataset', 'totalsegmentator',\n    '--data_root', str(datasets_root),\n    '--architecture', 'unetr',\n    '--in_channels', '1',\n    '--out_channels', '11',\n    '--batch_size', '1',\n    '--max_epochs', '50',\n    '--output_dir', 'results/unetr_totalsegmentator'\n], check=True)\n\nprint('UNETR + TotalSegmentator completed!')

## SegResNet + TotalSegmentator Training

In [ ]:
print(f'Starting SegResNet + TotalSegmentator at {datetime.now().strftime(\"%H:%M:%S\")}')\nprint('Expected duration: ~25 minutes')\nprint('=' * 50)\n\nsubprocess.run([\n    'python', 'scripts/train_model.py',\n    '--dataset', 'totalsegmentator',\n    '--data_root', str(datasets_root),\n    '--architecture', 'segresnet',\n    '--in_channels', '1',\n    '--out_channels', '11',\n    '--batch_size', '1',\n    '--max_epochs', '50',\n    '--output_dir', 'results/segresnet_totalsegmentator'\n], check=True)\n\nprint('SegResNet + TotalSegmentator completed!')

## Complete Training Summary

In [ ]:
# Check all training results across the entire project\nresults_dir = Path('results')\n\nall_experiments = [\n    'unet_brats', 'unet_msd_liver', 'unet_totalsegmentator',\n    'unetr_brats', 'unetr_msd_liver', 'unetr_totalsegmentator',\n    'segresnet_brats', 'segresnet_msd_liver', 'segresnet_totalsegmentator'\n]\n\nprint('Complete Training Status')\nprint('=' * 50)\n\ncompleted_count = 0\ntotal_size_mb = 0\n\nfor exp in all_experiments:\n    exp_dir = results_dir / exp\n    best_model = exp_dir / 'best.pth'\n    \n    if best_model.exists():\n        completed_count += 1\n        model_size = best_model.stat().st_size / (1024 * 1024)\n        total_size_mb += model_size\n        print(f'COMPLETED: {exp} ({model_size:.1f} MB)')\n    else:\n        print(f'NOT FOUND: {exp}')\n\nprint()\nprint(f'Final Progress: {completed_count}/{len(all_experiments)} experiments completed')\nprint(f'Total model size: {total_size_mb:.1f} MB')\n\nif completed_count == len(all_experiments):\n    print('\nAll experiments completed successfully!')\n    print('Ready for results analysis and final report.')\nelse:\n    remaining = len(all_experiments) - completed_count\n    print(f'\n{remaining} experiments remaining.')\n    print('Check previous notebooks if any experiments failed.')\n\nprint(f'\nSession completed at {datetime.now().strftime(\"%H:%M:%S\")}')